In [1]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(units=hp.Int("units", min_value=32, max_value=512, step=32),activation="relu"))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [2]:
import keras_tuner
build_model(keras_tuner.HyperParameters())

In [3]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(units=hp.Int("units", min_value=32, max_value=512, step=32),activation=hp.Choice("activation", ["relu", "tanh"])))
    # Tune whether to use dropout.
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),loss="categorical_crossentropy",metrics=["accuracy"])
    return model

build_model(keras_tuner.HyperParameters())

In [4]:
hp = keras_tuner.HyperParameters()
print(hp.Int("units", min_value=32, max_value=512, step=32))

32


In [5]:
def call_existing_code(units, activation, dropout, lr):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(units=units, activation=activation))
    if dropout:
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),loss="categorical_crossentropy",metrics=["accuracy"])
    return model

def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = call_existing_code(units=units, activation=activation, dropout=dropout, lr=lr)
    return model

build_model(keras_tuner.HyperParameters())

In [6]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(layers.Dense(units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),activation=hp.Choice("activation", ["relu", "tanh"])))
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),loss="categorical_crossentropy",metrics=["accuracy"])
    return model

build_model(keras_tuner.HyperParameters())

In [7]:
tuner = keras_tuner.RandomSearch(hypermodel=build_model,objective="val_accuracy",max_trials=3,executions_per_trial=2,overwrite=True,directory="my_dir",project_name="helloworld")
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [8]:
from tensorflow import keras
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 01m 44s]
val_accuracy: 0.9711999893188477

Best val_accuracy So Far: 0.9711999893188477
Total elapsed time: 00h 03m 06s
INFO:tensorflow:Oracle triggered exit


In [9]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 480)               376800    
                                                                 
 dense_1 (Dense)             (None, 448)               215488    
                                                                 
 dense_2 (Dense)             (None, 32)                14368     
                                                                 
 dense_3 (Dense)             (None, 10)                330       
                                                                 
Total params: 606,986
Trainable params: 606,986
Non-trainable params: 0
_________________________________________________________________


In [10]:
tuner.results_summary()

Results summary
Results in my_dir\helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 3
units_0: 480
activation: relu
dropout: False
lr: 0.0005372554068998837
units_1: 448
units_2: 32
Score: 0.9711999893188477

Trial 1 summary
Hyperparameters:
num_layers: 2
units_0: 320
activation: relu
dropout: False
lr: 0.005263426500053032
units_1: 128
Score: 0.965749979019165

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 288
activation: relu
dropout: False
lr: 0.0061221776185432925
units_1: 32
Score: 0.9636000096797943
